<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2023Q2/Un_primer_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum,get_min_max
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt





In [24]:
df_val=pd.read_csv('dataset/testear.csv')
df_train=pd.read_csv('dataset/origen.csv')
#split the train dataset into train and test
#df_train, df_test = train_test_split(df_train, test_size=0.1, random_state=421)

df_list = [df_train,df_val]



In [25]:
from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, MeanMedianImputer
for df in df_list:
    #turn numVotes into int
    df['numVotes'] = df['numVotes'].astype(int)
    df["dir_qty"] = pd.Series(df["directors"]).str.split(",").apply(len)
    df["writers_qty"] = pd.Series(df["writers"]).str.split(",").apply(len)
    
df_train = pd.DataFrame(AddMissingIndicator(variables=['seasonNumber','episodeNumber','language','runtimeMinutes',"genres_x",'status','tagline','video']).fit_transform(df_train))
df_val = pd.DataFrame(AddMissingIndicator(variables=['seasonNumber','episodeNumber','language','runtimeMinutes',"genres_x",'status','tagline','video']).fit_transform(df_val))
df_list = [df_train,df_val]


In [26]:
runtimeMean = df_train['runtimeMinutes'].quantile(0.5)
budgetMean = df_train['budget'].quantile(0.5)
revenueMean = df_train['revenue'].quantile(0.5)
votesMean = df_train['numVotes'].quantile(0.5)
#Get over the 3 quantile
runtimeQ3 = df_train['runtimeMinutes'].quantile(0.75)	
budgetQ3 = df_train['budget'].quantile(0.75)
revenueQ3 = df_train['revenue'].quantile(0.75)
votesQ3 = df_train['numVotes'].quantile(0.75)
dirMean = df_train['dir_qty'].quantile(0.5)
dirQ3 = df_train['dir_qty'].quantile(0.75)
writerMean = df_train['writers_qty'].quantile(0.5)
writerQ3 = df_train['writers_qty'].quantile(0.75)



for df in df_list:
    # flag if the value is over the mean
    df['runtimeMinutes_over_mean'] = np.where(df['runtimeMinutes']>runtimeMean, np.where(df['runtimeMinutes']>runtimeQ3,2,1), 0)
    df['budget_over_mean'] = np.where(df['budget']>budgetMean,np.where(df['budget']>budgetQ3,2,1), 0)
    df['revenue_over_mean'] = np.where(df['revenue']> revenueMean,np.where(df['revenue']>revenueQ3,2,1),0)
    df['numVotes_over_mean'] = np.where(df['numVotes']> votesMean,np.where(df['numVotes']>votesQ3,2,1),0)
    df['dir_qty_over_mean'] = np.where(df['dir_qty']> dirMean,np.where(df['dir_qty']>dirQ3,2,1),0)
    df['writers_qty_over_mean'] = np.where(df['writers_qty']> writerMean,np.where(df['writers_qty']>writerQ3,2,1),0)

del runtimeMean,budgetMean,revenueMean,votesMean,runtimeQ3,budgetQ3,revenueQ3,votesQ3

In [27]:

for df in df_list:
    # if isAdult is larger than 1, then set it to 1, if smaller that 0, then set it to 0
    df['isAdult'] = df['isAdult'].apply(lambda x: 1 if x > 1 else x)
    df['seasonNumber'] = df['seasonNumber'].fillna(0)
    df['episodeNumber'] = df['episodeNumber'].fillna(1)
    df['ordering'] = df['ordering'].fillna(0)
    df['language'] = df['language'].fillna(0)



In [28]:
#ADAPTADO A PIPELINE
# Completando los 0 con los promedios tomando en cuenta si son series, peliculas o episodios

for df in df_list:
    
    df['runtimeMinutes'] = df['runtimeMinutes'].replace(0, np.nan)
    df['runtimeMinutes'] = df.groupby('titleType')['runtimeMinutes'].apply(lambda x: x.interpolate(method='linear')).reset_index(level=0, drop=True)
    
    #Lo mismo pero para el budget
    df['budget'] = df['budget'].replace(0, np.nan)
    df['budget'] = df.groupby('titleType')['budget'].apply(lambda x: x.interpolate(method='linear')).reset_index(level=0, drop=True)
    
    # Si endYear es 0, entonces se le asigna el valor de startYear
    df['endYear'] = df.apply(lambda x: x['startYear'] if x['endYear'] == 0 else x['endYear'], axis=1)

    # Lo mismo pero para revenue
    df['revenue'] = df['revenue'].replace(0, np.nan)
    df['revenue'] = df.groupby('titleType')['revenue'].apply(lambda x: x.interpolate(method='linear')).reset_index(level=0, drop=True)


In [29]:
#ADAPTADO A PIPELINE
def get_exp_dict(df:pd.DataFrame,group:str,categories:str):
    categorias = df[categories].unique().tolist()
    categorias = [x.split(',') if type(x) == str else [] for x in categorias]
    categorias = [item for sublist in categorias for item in sublist]
    categorias = list(set(categorias))

    grupo = df.groupby(group)[categories].apply(list).to_dict()
    if '0' in grupo:
        del grupo['0']
    
    dict_limpio = dict()
    for director in grupo.keys():
        shows = grupo[director]
        
        for dir in director.split(','):
            if dir in dict_limpio:
                dict_limpio[dir] += shows
            else:
                dict_limpio[dir] = shows
    for director in dict_limpio.keys():
        #split all the genres into a list
        dict_limpio[director] = [x.split(',') if type(x) == str else [] for x in dict_limpio[director]]
        #flatten the list
        dict_limpio[director] = [item for sublist in dict_limpio[director] for item in sublist]
        #turn the list into a dict and count the number of times each genre appears
        dict_limpio[director] = dict((x,dict_limpio[director].count(x)) for x in set(dict_limpio[director]))
        # if not all genres are present, add the missing ones and set the count to 0
        for cat in categorias:
            if cat not in dict_limpio[director]:
                dict_limpio[director][cat] = 0
    return dict_limpio,categorias


def set_exp_by_cat(df:pd.DataFrame,group:str,dict_limpio:dict,categorias:list,tipo:str):
    for cat in categorias:
        df[f'{group}_exp_{cat}_{tipo}']=df['directors'].apply(sum_into_column,args=(dict_limpio,cat,))
    print('Done')

dirs_exp_genres,genres = get_exp_dict(df_train,'directors','genres_x')
writers_exp_genres,genres = get_exp_dict(df_train,'writers','genres_x')
dirs_exp_titleType,titleType = get_exp_dict(df_train,'directors','titleType')
writers_exp_titleType,titleType = get_exp_dict(df_train,'writers','titleType')

for df in df_list:
    set_exp_by_cat(df,'genres_x',dirs_exp_genres,genres,'dirs')
    set_exp_by_cat(df,'genres_x',writers_exp_genres,genres,'writers')
    set_exp_by_cat(df,'titleType',dirs_exp_titleType,titleType,'dirs')
    set_exp_by_cat(df,'titleType',writers_exp_titleType,titleType,'writers')
    
del dirs_exp_genres,writers_exp_genres,dirs_exp_titleType,writers_exp_titleType,genres,titleType



Done
Done
Done


C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\4185133187.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{group}_exp_{cat}_{tipo}']=df['directors'].apply(sum_into_column,args=(dict_limpio,cat,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\4185133187.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{group}_exp_{cat}_{tipo}']=df['directors'].apply(sum_into_column,args=(dict_limpio,cat,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\4185133187.py:37: PerformanceWarning: DataFrame is highly fragmented.  T

Done
Done
Done
Done


C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\4185133187.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{group}_exp_{cat}_{tipo}']=df['directors'].apply(sum_into_column,args=(dict_limpio,cat,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\4185133187.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{group}_exp_{cat}_{tipo}']=df['directors'].apply(sum_into_column,args=(dict_limpio,cat,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\4185133187.py:37: PerformanceWarning: DataFrame is highly fragmented.  T

Done


In [30]:

# Seasons worked

writers = df_train.groupby("writers").count()["Unnamed: 0"].to_dict()
writers = split_and_sum(writers)
seasons = df_train.groupby("writers").count()["seasonNumber"].to_dict()
seasons = split_and_sum(seasons)
episodes = df_train.groupby("writers").count()["episodeNumber"].to_dict()
episodes = split_and_sum(episodes)
movies = df_train[df_train["titleType"] == "movie"].groupby("writers").count()["titleType"].to_dict()
movies = split_and_sum(movies)
others = df_train[df_train["titleType"] != "movie"].groupby("writers").count()["titleType"].to_dict()
others = split_and_sum(others)

for df in df_list:
    
    


    df["directors_exp"] = df["directors"].apply(sum_into_column,args=(writers,))

    df["dir_minExperience"] = df["directors"].apply(get_min_max,args=(writers,0,))

    df["dir_maxExperience"] = df["directors"].apply(get_min_max,args=(writers,1,))

    df["dir_seasonsOfExperience"] = df["directors"].apply(sum_into_column,args=(seasons,))


    df["dir_episodesOfExperience"] = df["directors"].apply(sum_into_column,args=(episodes,))


    df["dir_moviesOfExperience"] = df["directors"].apply(sum_into_column,args=(movies,))

    df["dir_othersOfExperience"] = df["directors"].apply(sum_into_column,args=(others,))

    
    
    

del writers,seasons,episodes,movies,others

C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\169548390.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["directors_exp"] = df["directors"].apply(sum_into_column,args=(writers,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\169548390.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["dir_minExperience"] = df["directors"].apply(get_min_max,args=(writers,0,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\169548390.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling 

In [31]:
writers_exp = df_train.groupby("writers").count()["Unnamed: 0"].to_dict()
writers_exp = split_and_sum(writers_exp)
seasons = df_train.groupby("writers").count()["seasonNumber"].to_dict()
seasons = split_and_sum(seasons)
episodes = df_train.groupby("writers").count()["episodeNumber"].to_dict()
episodes = split_and_sum(episodes)
movies = df_train[df_train["titleType"] == "movie"].groupby("writers").count()["titleType"].to_dict()
movies = split_and_sum(movies)
others = df_train[df_train["titleType"] != "movie"].groupby("writers").count()["titleType"].to_dict()
others = split_and_sum(others)


for df in df_list:


    df["writers_exp"] = df["writers"].apply(sum_into_column,args=(writers_exp,))

    df["writer_minExperience"] = df["writers"].apply(get_min_max,args=(writers_exp,0,))

    df["writer_maxExperience"] = df["writers"].apply(get_min_max,args=(writers_exp,1,))

    df["writer_seasonsOfExperience"] = df["writers"].apply(sum_into_column,args=(seasons,))
    
    df["writer_episodesOfExperience"] = df["writers"].apply(sum_into_column,args=(episodes,))

    df["writer_moviesOfExperience"] = df["writers"].apply(sum_into_column,args=(movies,))

    df["writer_othersOfExperience"] = df["writers"].apply(sum_into_column,args=(others,))


del seasons,episodes,movies,others,writers_exp

C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\2747713769.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["writers_exp"] = df["writers"].apply(sum_into_column,args=(writers_exp,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\2747713769.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["writer_minExperience"] = df["writers"].apply(get_min_max,args=(writers_exp,0,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\2747713769.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of 

In [32]:
def parse_json(x):
    x = x.split(":")
    x = x[1:]
    for i in range(len(x)):
        x[i] = x[i].split("}")[0]
        x[i] = x[i].split(",")[0]
    
    #split the list into pairs of two
    x = [x[i:i+2] for i in range(0,len(x),2)]
    # grab the first element of the pair
    x = [i[0] for i in x]
    #remove the quotes
    x = [i.replace("'",'') for i in x]
    #remove the spaces
    x = [i.strip() for i in x]

    return ",".join(x)

for df in df_list:
    df["production_countries"] = df["production_countries"].fillna("[]")
    #turn str list into list
    df["production_countries"] = df["production_countries"].apply(parse_json)

    df["production_companies"] = df["production_companies"].fillna("[]")
    #turn str list into list
    df["production_companies"] = df["production_companies"].apply(parse_json)

experience = df_train.groupby("genres_x").count()["Unnamed: 0"].to_dict()
experience['']=0

companies_experience = split_and_sum(df.groupby("production_companies").count()["Unnamed: 0"].to_dict())
companies_experience['']=0

dir_votes = df_train.groupby("directors")["numVotes"].sum().to_dict()
dir_votes = split_and_sum(dir_votes)

for df in df_list:

    df["production_countries_experience"] = df["production_countries"].apply(sum_into_column,args=(experience,))

    df['production_countries'] = df['production_countries'].apply(lambda x: len(x.split(',')))

    #turn production company into a list
    #fill nans with empty list


    df["production_companies_experience"] = df["production_companies"].apply(sum_into_column,args=(companies_experience,))

    df['production_companies'] = df['production_companies'].apply(lambda x: len(x.split(',')))
    
    df["dir_votes"] = df["directors"].apply(sum_into_column,args=(dir_votes,))
    
del experience,companies_experience,dir_votes

C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\3410312861.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["production_countries_experience"] = df["production_countries"].apply(sum_into_column,args=(experience,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\3410312861.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["production_companies_experience"] = df["production_companies"].apply(sum_into_column,args=(companies_experience,))
C:\Users\Ian\AppData\Local\Temp\ipykernel_14396\3410312861.py:51: PerformanceWarni

In [33]:
# turn video colunm into 1 or 0
# video column might have False, True or NaN values
for df in df_list:
    df["video"] = df["video"].fillna(False)
    df["video"] = df["video"].astype(int)
    df["tagline"] = df["tagline"].fillna("")
    df["tagline"] = df["tagline"].apply(len)



In [34]:
y=df_train.averageRating
vars=['Unnamed: 0',  'titleType',
       'genres_x', 'directors',
       'writers','language',
       'attributes', 'genres_y',
        'production_companies',
       'production_countries', 'adult',
       'original_language', 'runtime', 'status']


         # 'revenue' Evaluar si sirve o no
       
       
X_pred = df_val.drop(columns= vars)
X = df_train.drop(columns= vars)
#X_test=df_test[vars]

for db in [[X,df_train],[X_pred,df_val]]:
      db[0] = pd.concat([db[0], pd.get_dummies(db[1]['language'], prefix='language')], axis=1)
      db[0] = pd.concat([db[0], pd.get_dummies(db[1]['status'], prefix='status')], axis=1)

del db



In [35]:
for i,df in enumerate(df_list):
    encoder = OneHotEncoder()
    title = encoder.fit_transform(df[['titleType']])
    title_df = pd.DataFrame(title.toarray(), columns=encoder.get_feature_names_out(['titleType']))
    #split by genres
    genres = df.genres_x.str.get_dummies(sep=',')
    genres = genres.reindex(sorted(genres.columns), axis=1)

    title_df = pd.concat([title_df, genres], axis=1)
    if i==0:
        X = pd.concat([X, title_df], axis=1)
    elif i==1:
        X_pred = pd.concat([X_pred, title_df], axis=1)
    print(i,"Done!")
    


del title_df, genres, title, encoder, df,i





0 Done!
1 Done!


In [36]:
#Drop categorical columns but print them first
for df in [X,X_pred]:
    print(df.select_dtypes(include=['object']).columns.tolist())
    df.drop(columns=df.select_dtypes(include=['object']).columns.tolist(),inplace=True)

# Drop duplicate columns
X = X.loc[:, ~X.columns.duplicated()]
X_pred = X_pred.loc[:, ~X_pred.columns.duplicated()]


# Add missing columns to X_pred and X_test
missing_cols = set(X.columns) - set(X_pred.columns)
X_pred = X_pred.reindex(columns=X_pred.columns.tolist() + list(missing_cols))


# Add missing columns to X
missing_cols = set(X_pred.columns) - set(X.columns)
X = X.reindex(columns=X.columns.tolist() + list(missing_cols))



[]
[]


In [37]:
X_pred = X_pred.sort_index(axis=1)
X = X.sort_index(axis=1)


#Drop averageRating from X_pred
X_pred = X_pred.drop(columns=["averageRating"])

X.to_csv('dataset/train.csv', index=False)
X_pred.to_csv('dataset/val.csv', index=False)
